# Midterm 2

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 2` assignment on Canvas, where you downloaded this. 
* Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers.)
* Your submission should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo - you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file contains the following sheets:
- for Section 2:
    * `sector stocks excess returns` - MONTHLY excess returns for 49 sector stocks
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5
- for Section 3:
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 25     |
| 2       | 40     |
| 3       | 35     |

### Each numbered question is worth 5 points unless otherwise specified.

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.1.

Historically, which pricing factor among the ones we studied has shown a considerable decrease in importance?

The SMB pricing factor has shown considerable decrease in importance as mentioned by Mark in lecture. From the HW it is observed that the HML factor is highly correlated with CMA and RMW, indicating that is might have less importance. 

### 1.2.

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the cross-sectional MAE. 

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the time-series MAE. 

Along with stating T/F, explain your reasoning for the two statements.

True for first, cross sectional regression, there is no risk free rate assumption to begin with in the model, we are trying to explain the expected excess returns with factor beta. Having more factors would increase the variance explained and reduce and form of error or unexplained portion (alpha).

False for second. The construction of the time series regression by fitting excess over risk free rate time series regression against factors, forces the regression line to fit at the intercept (risk free rate) which is expected to be 0, adding more factors does not necessarily improve the regression alpha by forcing it to fit the risk free rate.

### 1.3.

Consider the scenario in which you are helping two people with investments.

* The young person has a 50 year investment horizon.
* The elderly person has a 10 year investment horizon.
* Both individuals have the same portfolio holdings.

State who has the more certain cumulative return and explain your reasoning.

Cumulative returns variance scales with time, meaning that the young person's cumulative returns definetley have a higher varaince than that of a person with a 10 year investment horizon. So elderly have a more certain cumulative returns.

### 1.4.

Suppose we find that the 10-year bond yield works well as a new pricing factor, along with `MKT`.

Consider two ways of building this new factor.
1. Directly use the index of 10-year yields, `YLD`
1. Construct a Fama-French style portfolio of equities, `FFYLD`. (Rank all the stocks by their correlation to bond yield changes, and go long the highest ranked and shor tthe lowest ranked.)

Could you test the model with `YLD` and the model with `FFYLD` in the exact same ways? Explain.

Testing the first version, cant use a multivariate time series regresstion with mkt as second factor as the OLS esimate is baised. Variance inflation factor would likely be high due to high correlation with market. Cross sectional regression might be the most appropriate, or using a tangency portfolio to see if the weights are resonable. On the other hand, second version already hedged out effects of market, a time series test is reasonable as there is less collinearity, followed by cross sectional to check R squared again. 

### 1.5.

Suppose we implement a momentum strategy on cryptocurrencies rather than US stocks.

Conceptually speaking, but specific to the context of our course discussion, how would the risk profile differ from the momentum strategy of US equities?

Crypto assets are more volatile compared to US equities, price movements more pronounded, therefore the auto correlation should be bigger than that of US equities, meaning we might not need to rely on selection effect as much as compared to US equities, maybe one or 2 Crypto assets instead of top deciles and bottom deciles that is used in US equites. Taking mote crypto assets might instead make the portfolio very very risky due to the much higher returns

***

# 2. Pricing and Tangency Portfolio

You work in a hedge fund that believes that the AQR 4-Factor Model (present in Homework 5) is the perfect pricing model for stocks.

$$
\mathbb{E} \left[ \tilde{r}^i \right] = \beta^{i,\text{MKT}} \mathbb{E} \left[ \tilde{f}_{\text{MKT}} \right] + \beta^{i,\text{HML}} \mathbb{E} \left[ \tilde{f}_{\text{HML}} \right] + \beta^{i,\text{RMW}} \mathbb{E} \left[ \tilde{f}_{\text{RMW}} \right] + \beta^{i,\text{UMD}} \mathbb{E} \left[ \tilde{f}_{\text{UMD}} \right]
$$

The factors are available in the sheet `factors excess returns`.

The hedge fund invests in sector-tracking ETFs available in the sheet `sectors excess returns`. You are to allocate into these sectors according to a mean-variance optimization with...

* regularization: elements outside the diagonal covariance matrix divided by 2.
* modeled risk premia: expected excess returns given by the factor model rather than just using the historic sample averages.

You are to train the portfolio and test out-of-sample. The timeframes should be:
* Training timeframe: Jan-2018 to Dec-2022.
* Testing timeframe: Jan-2023 to most recent observation.

In [1]:
import numpy as np
import os
from arch import arch_model
import re
import sys
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
import sys
from typing import Tuple, Dict, Union, Callable
notebook_dir = "Exam"
if os.path.basename(os.getcwd()) == notebook_dir:
    os.chdir(os.pardir)  # Change to parent directory
    sys.path.append(os.getcwd())  # Add the new current directory to sys.path

import cmds.portfolio_management_helper as pmh
import functions.functions_andy as fa
print(os.getcwd())

from sklearn.linear_model import LinearRegression
from dask.distributed import Client, wait
# From HW1
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.metrics import r2_score
from scipy.stats import describe



c:\Users\Andik\OneDrive\Desktop\Chicago\Portfolio theory and risk management\githubrepo\finm-portfolio-2024


### 2.1.
(8pts)

Calculate the model-implied expected excess returns of every asset.

The time-series estimations should...
* NOT include an intercept. (You assume the model holds perfectly.)
* use data from the `training` timeframe.

With the time-series estimates, use the `training` timeframe's sample average of the factors as the factor premia. Together, this will give you the model-implied risk premia, which we label as
$$
\lambda_i := \mathbb{E}[\tilde{r}_i]
$$

* Store $\lambda_i$ and $\boldsymbol{\beta}^i$ for each asset.
* Print $\lambda_i$ for `Agric`, `Food`, `Soda`

In [5]:
factors_data = fa.load_data(filename="midterm_2_data.xlsx",sheet_name="factors excess returns")
sector_data = fa.load_data(filename="midterm_2_data.xlsx",sheet_name="sector excess returns")

factors_data.set_index("date",inplace=True)
sector_data.set_index("date",inplace=True)
display(factors_data.head())
display(sector_data.head())

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.0551,0.0175,-0.0170,0.0755
1980-02-01,-0.0122,0.0061,0.0004,0.0788
1980-03-01,-0.1290,-0.0101,0.0146,-0.0955
1980-04-01,0.0397,0.0106,-0.0210,-0.0043
1980-05-01,0.0526,0.0038,0.0034,-0.0112


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,-0.0076,0.0285,0.0084,0.1009,-0.0143,0.1002,0.0362,0.0323,0.0048,0.0059,...,0.0158,0.0875,0.0465,-0.0126,0.0430,-0.0283,0.0258,0.0768,0.0308,0.0669
1980-02-01,0.0105,-0.0608,-0.0966,-0.0322,-0.0569,-0.0323,-0.0521,-0.0800,-0.0555,-0.0167,...,-0.0079,-0.0541,-0.0346,-0.0639,-0.0652,-0.0854,-0.0959,-0.0347,-0.0282,-0.0274
1980-03-01,-0.2224,-0.1119,-0.0167,-0.1469,-0.0193,-0.1271,-0.0826,-0.1237,-0.0566,-0.0668,...,-0.0819,-0.1509,-0.1098,-0.0906,-0.1449,-0.0560,-0.0880,-0.2451,-0.1254,-0.1726
1980-04-01,0.0449,0.0766,0.0232,0.0321,0.0830,-0.0529,0.0783,0.0153,0.0304,0.0115,...,0.0420,-0.0103,-0.0312,0.0353,0.0542,0.0728,0.0530,0.0977,0.0447,0.0769
1980-05-01,0.0632,0.0793,0.0457,0.0863,0.0815,0.0509,0.0324,0.0886,0.0560,0.0098,...,0.0564,0.1063,0.1142,0.0877,0.1134,0.0578,0.0557,0.0915,0.0844,0.0685


In [8]:
train_sector = sector_data.loc[pd.to_datetime("2018-01-01"):pd.to_datetime("2022-12-31")].copy()
test_sector = sector_data.loc[pd.to_datetime("2023-01-01"):].copy()
train_factors = factors_data.loc[pd.to_datetime("2018-01-01"):pd.to_datetime("2022-12-31")].copy()
test_factors = factors_data.loc[pd.to_datetime("2023-01-01"):].copy()

In [37]:
results = {}
for col in sector_data.columns:
    y = train_sector[col].values.reshape(-1,1)
    X = train_factors.values
    model = LinearRegression().fit(X,y)
    betas = model.coef_.flatten()
    beta_df = pd.DataFrame(betas, index=train_factors.columns).T
    beta_df["alpha"] = model.intercept_
    results[col] = beta_df.T

results_df = pd.concat(results, axis=1)
print("Betas and alpha for each sector from time series regression")
results_df

Betas and alpha for each sector from time series regression


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MKT,0.8090,0.5277,0.5337,0.5416,0.5056,1.4246,1.3734,1.0852,0.4782,1.1326,...,0.7839,1.1256,0.9808,1.0168,0.8980,1.0637,0.7695,1.2747,1.0626,0.8727
HML,0.5546,0.2057,0.1786,0.0230,0.4437,-0.1189,-0.1862,0.2799,-0.0945,-0.1715,...,0.0201,0.1470,0.2587,-0.3727,0.1110,0.5925,0.3032,0.3667,0.2494,0.3313
RMW,-0.5621,0.3178,0.6217,0.6305,0.4214,-0.9659,-0.8567,-0.3946,0.8476,0.3459,...,0.3144,0.0265,0.2015,0.0889,0.2939,-0.2642,0.0449,-0.3958,-0.1089,0.0182
UMD,0.0017,0.0015,0.0033,-0.0400,-0.1221,0.1547,-0.0487,-0.3999,-0.0744,-0.2078,...,-0.1171,-0.1164,-0.0618,0.0784,-0.0345,-0.1990,0.0245,-0.5314,-0.0898,-0.0214
alpha,0.0071,-0.0010,0.0020,-0.0001,-0.0023,-0.0057,-0.0011,-0.0021,-0.0004,-0.0004,...,-0.0031,-0.0040,0.0026,-0.0001,0.0005,-0.0003,0.0046,-0.0002,0.0021,-0.0001


In [68]:
beta_data = results_df.copy()
beta_data.columns = beta_data.columns.droplevel(1)
estimated_results = {}
for col in beta_data.columns:
    betas = beta_data[col].values[:-1]
    alphas = beta_data[col].values[-1]
    mean_factors = train_factors.mean().values
    estimated_premia = np.dot(betas, mean_factors) + alphas
    estimated_results[col] = estimated_premia
    
print("Estimated premia for each sector")
estimated_results_df = pd.DataFrame(estimated_results, index=["Estimated Premia"]).T
estimated_results_df

Estimated premia for each sector


,Estimated Premia
Agric,0.0102
Food,0.0046
Soda,0.0092
Beer,0.0070
Smoke,0.0031
Toys,0.0007
Fun,0.0047
Books,0.0024
Hshld,0.0073
Clths,0.0092


In [71]:
# Annualised expected premia  for Agric, Food, Soda
annualised_estimated_premia_subset = estimated_results_df.loc[["Agric", "Food ", "Soda "]].copy() * 12
display(annualised_estimated_premia_subset)

,Estimated Premia
Agric,0.1220
Food,0.0549
Soda,0.1098


## Note Below is sample historical estimate for my reference, above is the answer for 2.1

In [41]:
# Sample average of train as expected return of each asset
expected_returns = train_sector.mean()
expected_returns = expected_returns.to_frame(name="Expected Mean").T
display(expected_returns)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Expected Mean,0.0102,0.0046,0.0092,0.0070,0.0031,0.0007,0.0047,0.0024,0.0073,0.0092,...,0.0039,0.0041,0.0107,0.0085,0.0086,0.0053,0.0107,0.0051,0.0091,0.0063


In [46]:
# Print expected mean for Agric, Food, Soda
# print("Expected mean for Agric, Food, Soda")
# display(expected_returns.loc[:,["Agric","Food","Soda"]])
expected_returns.loc[:,["Agric","Food ","Soda "]]

# Annaulised
annualised_expected_returns = expected_returns.loc[:,["Agric","Food ","Soda "]] * 12
print("Annualised")
display(annualised_expected_returns)

Annualised


,Agric,Food,Soda
Expected Mean,0.1220,0.0549,0.1098


### 2.2.

Use the expected excess returns derived from (2.1) with the **regularized** covariance matrix to calculate the weights of the tangency portfolio.

- Use the covariance matrix only for `training` timeframe.
- Calculate and store the vector of weights for all the assets.
- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

$$
\textbf{w}_{t} = \dfrac{\tilde{\Sigma}^{-1} \bm{\lambda}}{\bm{1}' \tilde{\Sigma}^{-1} \bm{\lambda}}
$$

Where $\tilde{\Sigma}^{-1}$ is the regularized covariance-matrix.

In [79]:
annual_factor =12
cov_matrix = train_sector.cov() * annual_factor
estimated_mean_premia = estimated_results_df* 12
weights_estimated = np.linalg.inv(cov_matrix).dot(estimated_mean_premia.values) / np.sum(np.linalg.inv(cov_matrix).dot(estimated_mean_premia.values))
weights_estimated_df = pd.DataFrame(weights_estimated, index=estimated_mean_premia.index, columns=["Weight"])
weights_estimated_df

,Weight
Agric,-1.0443
Food,-0.8693
Soda,15.3635
Beer,-6.7836
Smoke,-0.7639
Toys,-2.8105
Fun,1.1799
Books,5.0898
Hshld,13.3243
Clths,6.7487


In [80]:
# weights for Agric, food soda
weights_estimated_df.loc[["Agric", "Food ", "Soda "]]
display(weights_estimated_df.loc[["Agric", "Food ", "Soda "]])


,Weight
Agric,-1.0443
Food,-0.8693
Soda,15.3635


### 2.3.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [82]:
# test period stats
portfolio_expected_mean = (weights_estimated_df.T @ test_sector.mean().T).T * annual_factor
portfolio_volatility = np.sqrt(weights_estimated_df.T @ test_sector.cov() @ weights_estimated_df) * np.sqrt(annual_factor)
portfolio_sharpe = portfolio_expected_mean / portfolio_volatility
print("Portfolio expected mean in test set")
display(portfolio_expected_mean.values)
print("Portfolio volatility in test set")
display(portfolio_volatility.values)
print("Portfolio Sharpe in test set")
display(portfolio_sharpe.values)

Portfolio expected mean in test set


array([-1.77314208])

Portfolio volatility in test set


array([[7.33079103]])

Portfolio Sharpe in test set


array([[-0.24187595]])

### 2.4.

(7pts)

Construct the same tangency portfolio as in `2.2` but with one change:
* replace the risk premia of the assets, (denoted $\lambda_i$) with the sample averages of the excess returns from the `training` set.

So instead of using $\lambda_i$ suggested by the factor model (as in `2.1-2.3`) you're using sample averages for $\lambda_i$.

- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [54]:
annual_factor =12
cov_matrix = train_sector.cov() * annual_factor
expected_returns_annualised = (expected_returns * annual_factor).T
weights = np.linalg.inv(cov_matrix) @ expected_returns_annualised / (np.ones(len(expected_returns_annualised)).T @ np.linalg.inv(cov_matrix) @ expected_returns_annualised)
weights.index = expected_returns.columns
weights.columns= ["Weights"]
display(weights)
display(weights.loc[["Agric","Food ","Soda "]])

,Weights
Agric,-1.0443
Food,-0.8693
Soda,15.3635
Beer,-6.7836
Smoke,-0.7639
Toys,-2.8105
Fun,1.1799
Books,5.0898
Hshld,13.3243
Clths,6.7487


,Weights
Agric,-1.0443
Food,-0.8693
Soda,15.3635


In [58]:
# Portfolio expected mean in test set
portfolio_expected_mean = (weights.T @ test_sector.mean().T).T * annual_factor
portfolio_volatility = np.sqrt(weights.T @ test_sector.cov() @ weights) * np.sqrt(annual_factor)
portfolio_sharpe = portfolio_expected_mean / portfolio_volatility
print("Portfolio expected mean in test set")
display(portfolio_expected_mean.values)
print("Portfolio volatility in test set")
display(portfolio_volatility.values)
print("Portfolio Sharpe in test set")
display(portfolio_sharpe.values)



Portfolio expected mean in test set


array([-1.77314208])

Portfolio volatility in test set


array([[7.33079103]])

Portfolio Sharpe in test set


array([[-0.24187595]])

### 2.5.

Which allocation performed better in the `testing` period: the allocation based on premia from the factor model or from the sample averages?

Why might this be?

The utilization of risk premia estimated is better as mean sample estimates are less accurate. 

### 2.6.
Suppose you now want to build a tangency portfolio solely from the factors, without using the sector ETFs.

- Calculate the weights of the tangency portfolio using `training` data for the factors.
- Again, regularize the covariance matrix of factor returns by dividing off-diagonal elements by 2.

Report, in the `testing` period, the factor-based tangency stats **annualized**...
- mean
- vol
- Sharpe


In [85]:
# weights from factors
factor_mean_port = train_factors.mean()
factor_cov_port = train_factors.cov()
weights_factor = np.linalg.inv(factor_cov_port) @ factor_mean_port / np.sum(np.linalg.inv(factor_cov_port) @ factor_mean_port)

weights_factor_df = pd.DataFrame(weights_factor, index=train_factors.columns, columns=["Weights"])
display(weights_factor_df)

# test stats
portfolio_expected_mean = (weights_factor_df.T @ test_factors.mean().T).T * annual_factor
portfolio_volatility = np.sqrt(weights_factor_df.T @ test_factors.cov() @ weights_factor_df) * np.sqrt(annual_factor)
portfolio_sharpe = portfolio_expected_mean / portfolio_volatility
print("Portfolio expected mean in test set")
display(portfolio_expected_mean.values)
print("Portfolio volatility in test set")
display(portfolio_volatility.values)
print("Portfolio Sharpe in test set")
display(portfolio_sharpe.values)

,Weights
MKT,0.1905
HML,0.0158
RMW,0.4894
UMD,0.3043


Portfolio expected mean in test set


array([0.05271698])

Portfolio volatility in test set


array([[0.06069967]])

Portfolio Sharpe in test set


array([[0.86848873]])

### 2.7.

Based on the hedge fund's beliefs, would you prefer to use the ETF-based tangency or the factor-based tangency portfolio? Explain your reasoning. Note that you should answer based on broad principles and not on the particular estimation results.

If the hedge fund prioritizes targeted risk exposure and believes in the potential of factor investing to achieve superior risk-adjusted returns like DFA, then the factor-based tangency portfolio would be the preferred choice. However, if the fund values broad market exposure, then the ETF-based tangency portfolio would be preferable. The factor based tangency portfolio definetely performs the best in tet sets as shown above.



***

# 3. Long-Run Returns

For this question, use only the sheet `factors excess returns`.

Suppose we want to measure the long run returns of various pricing factors.

### 3.1.

Turn the data into log returns.
- Display the first 5 rows of the data.

Using these log returns, report the **annualized**
* mean
* vol
* Sharpe

In [19]:
log_factor_data = factors_data.apply(np.log1p)
display(log_factor_data.head())

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.0536,0.0173,-0.0171,0.0728
1980-02-01,-0.0123,0.0061,0.0004,0.0758
1980-03-01,-0.1381,-0.0102,0.0145,-0.1004
1980-04-01,0.0389,0.0105,-0.0212,-0.0043
1980-05-01,0.0513,0.0038,0.0034,-0.0113


In [22]:
# Report stats
annual_factor = 12
mean_data = log_factor_data.mean()
std_data = log_factor_data.std()
sharp_data = mean_data/std_data
annualised_mean_data = mean_data*annual_factor
annualised_std_data = std_data*np.sqrt(annual_factor)
annualised_sharp_data = sharp_data*np.sqrt(annual_factor)
summary_stats = pd.concat([mean_data,std_data,sharp_data,annualised_mean_data,annualised_std_data,annualised_sharp_data],axis=1)
summary_stats.columns = ["Mean","Std","Sharp","Annualised Mean","Annualised Std","Annualised Sharp"]
display(summary_stats)


,Mean,Std,Sharp,Annualised Mean,Annualised Std,Annualised Sharp
MKT,0.0061,0.0459,0.1337,0.0735,0.1588,0.4630
HML,0.0016,0.0317,0.0520,0.0198,0.1098,0.1801
RMW,0.0036,0.0241,0.1504,0.0435,0.0836,0.5210
UMD,0.0042,0.0463,0.0901,0.0501,0.1604,0.3122




### 3.2.

Consider 15-year cumulative log excess returns. Following the assumptions and modeling of Lecture 6, report the following 15-year stats:
- mean
- vol
- Sharpe

How do they compare to the estimated stats (1-year horizon) in `3.1`? 

In [23]:
# 15 year cumulative log excess returns, assume normal distribution
log_cumulative_returns = log_factor_data.cumsum()
log_mean = log_cumulative_returns.mean()
log_std = log_cumulative_returns.std()
log_sharp = log_mean/log_std
log_annualised_mean = log_mean*annual_factor
log_annualised_std = log_std*np.sqrt(annual_factor)
log_annualised_sharp = log_sharp*np.sqrt(annual_factor)
log_summary_stats = pd.concat([log_mean,log_std,log_sharp,log_annualised_mean,log_annualised_std,log_annualised_sharp],axis=1)
log_summary_stats.columns = ["Mean","Std","Sharp","Annualised Mean","Annualised Std","Annualised Sharp"]
display(log_summary_stats)

,Mean,Std,Sharp,Annualised Mean,Annualised Std,Annualised Sharp
MKT,1.4759,0.8920,1.6545,17.7105,3.0901,5.7313
HML,0.7876,0.3658,2.1533,9.4511,1.2670,7.4592
RMW,0.9804,0.5420,1.8089,11.7654,1.8776,6.2663
UMD,1.6420,0.7445,2.2056,19.7036,2.5789,7.6403


Evidently, the standard deviations of the cumlative returns are much larger in the 15 year horizon. The increase in volatility in the cumulative returns data is also evident in the sharp statistics. The mean cumulative returns are also larger in the cumulative returns.

### 3.3.

What is the probability that momentum factor has a negative mean excess return over the next 
* single period?
* 15 years?

In [28]:
from scipy.stats import norm
## Single period prob
momentum_factor_mean = summary_stats.loc["UMD","Annualised Mean"]
momentum_factor_std = summary_stats.loc["UMD","Annualised Std"]
prob = norm.cdf(-momentum_factor_mean/momentum_factor_std)
display(f"Probability that momentum factor has a negative mean excess return over the next single period is {prob}")

## 15 years prob
momentum_factor_mean = summary_stats.loc["UMD","Annualised Mean"]
momentum_factor_std = summary_stats.loc["UMD","Annualised Std"]
prob = norm.cdf(-momentum_factor_mean*np.sqrt(15)/momentum_factor_std)
display(f"Probability that momentum factor has a negative mean excess return over the next 15 years is {prob}")

'Probability that momentum factor has a negative mean excess return over the next single period is 0.37742600346334504'

'Probability that momentum factor has a negative mean excess return over the next 15 years is 0.11326774683663704'

### 3.4.

Recall from the case that momentum has been underperforming since 2009. 

Using data from 2009 to present, what is the probability that momentum *outperforms* the market factor over the next
* period?
* 15 years?

In [32]:
factors_data_2009 = log_factor_data.loc[pd.to_datetime("2009-01-01"):].copy()
factors_data_2009["mom-mkt"] = factors_data_2009["UMD"] - factors_data_2009["MKT"]
factors_data_2009_mean = factors_data_2009.mean()
factors_data_2009_std = factors_data_2009.std()
factors_data_2009_sharp = factors_data_2009_mean/factors_data_2009_std
factors_data_2009_annualised_mean = factors_data_2009_mean*annual_factor
factors_data_2009_annualised_std = factors_data_2009_std*np.sqrt(annual_factor)
factors_data_2009_annualised_sharp = factors_data_2009_sharp*np.sqrt(annual_factor)
factors_data_2009_summary_stats = pd.concat([factors_data_2009_mean,factors_data_2009_std,factors_data_2009_sharp,factors_data_2009_annualised_mean,factors_data_2009_annualised_std,factors_data_2009_annualised_sharp],axis=1)
factors_data_2009_summary_stats.columns = ["Mean","Std","Sharp","Annualised Mean","Annualised Std","Annualised Sharp"]
display(factors_data_2009_summary_stats)

mom_mkt_mean_2009 = factors_data_2009_summary_stats.loc["mom-mkt","Annualised Mean"]
mom_mkt_std_2009 = factors_data_2009_summary_stats.loc["mom-mkt","Annualised Std"]
# One period prob
prob = 1 -norm.cdf(-mom_mkt_mean_2009/mom_mkt_std_2009)
display(f"Probability that momentum factor outperforming mkt over the next single period is {prob}")
# 15 years prob
prob = 1- norm.cdf(-mom_mkt_mean_2009*np.sqrt(15)/mom_mkt_std_2009)
display(f"Probability that momentum factor outperforming mkt over the next 15 years is {prob}")



,Mean,Std,Sharp,Annualised Mean,Annualised Std,Annualised Sharp
MKT,0.0105,0.0453,0.2321,0.1261,0.1568,0.8041
HML,-0.0018,0.0346,-0.0525,-0.0218,0.1200,-0.1818
RMW,0.0028,0.0194,0.1422,0.0331,0.0673,0.4925
UMD,-0.0024,0.0497,-0.0482,-0.0287,0.1723,-0.1669
mom-mkt,-0.0129,0.0782,-0.1649,-0.1548,0.2710,-0.5713


'Probability that momentum factor outperforming mkt over the next single period is 0.28389932344697144'

'Probability that momentum factor outperforming mkt over the next 15 years is 0.013461859038858881'

### 3.5.
Conceptually, why is there such a discrepancy between this probability for 1 period vs. 15 years?

What assumption about the log-returns are we making when we use this technique to estimate underperformance?

The disrepancy between probabilities between 1 period and 15 period is due to the notion of time diversification, where returns are lognormally distributed and iid. With the assumption of no serial correlation between returns as well, then the expected returns E[x] = mean return regardless of 1 period or 15 period, but the variance (sample) is given Var(x) = annualised_standard_deviation/period. Meaning that variance decreases over time by law of large numbers, and the probability of having a certain expected mean return becomes almost certain in the long run, making it almost riskless to hold an asset over a long period of time, and thus prob of underperforming falls with time under these assumptions

### 3.6.

Using your previous answers, explain what is meant by time diversification.

Time diversifcation therefore, is a way to reduce the risk of the portfolio by investing in the long run. As the variance scales with sqrt(period) and mean scales with period, making the overall sharpe ratio much higher with time. With time diversifcation, it is assumed that one can achieve close to riskless portfolio by holding a single asset over the long run.

### 3.7.

Is the probability that `HML` and `UMD` both have negative cumulative returns over the next year higher or lower than the probability that `HML` and `MKT` both have negative cumulative returns over the next year?

Answer conceptually, but specifically. (No need to calculate the specific probabilities.)

UMD has a lower sharpe than MKT, meaning the z-score of 0 would be less negative than mkt, thus the cumulative distribution function at the scaled point of 0 would be greater than that of the market. Thus the prob of underperforming the risk free rate is greater than thay of the market. Assuming independence, than the prob of HML and prob(UMD) having negative cumulative returns can be calculated using multiplicative rules of the 2 probabilities, same as the case of HML and MKT. This would mean that the Prob of HML and UMD both have negative cumulative returns over the next year would be higher than that of HML and MKT 

***